In [1]:
import torch
import numpy as np

from datasets import VOCDectectionDataset
from sklearn.metrics import average_precision_score
from model import *
from utils import *
from tqdm.notebook import tqdm


In [5]:
propose_way = "edge_box"
pretrained = "vgg16"
traindata = VOCDectectionDataset("~/data/", 2007, 'trainval', region_propose=propose_way, do_transform=True)
traindata_loader = data.DataLoader(traindata, 1, shuffle=False)
wsddn = WSDDN_VGG16().to(DEVICE)
wsddn.load_state_dict(torch.load(SAVE_PATH + get_model_name(propose_way, "2007" , f"wsddn_{pretrained}") + ".pt"))

<All keys matched successfully>

In [6]:
positives = np.array([0 for _ in range(20)]) # 
hits = np.array([0 for _ in range(20)]) # 
n = 0

with torch.no_grad():
    wsddn.eval();
    for img, gt_box, gt_target, regions, scores in tqdm(traindata_loader, "CorLoc"):
        n += 1
        regions = regions.to(DEVICE)
        if propose_way != "edge_box":
            scores = None
        else:
            scores = scores.to(DEVICE)
        img = img.to(DEVICE)
        regions = regions.to(DEVICE)
        gt_box = gt_box.to(DEVICE)
        combined_scores, fc7= wsddn(img, regions, scores)
        
        for c in range(20):
            if gt_target[0][c] == 0.0:
                continue
            else:
                positives[c] += 1
                gt_regions = gt_box[0]
                cls_scores = combined_scores[:, c]
                most_confident_one = torch.argmax(cls_scores)
                most_confident_region = regions[0][most_confident_one:most_confident_one+1]
                IOU = one2allbox_iou(most_confident_region, gt_regions)
                if (IOU >= 0.5).sum() >= 1.0:
                    hits[c] += 1
        if n % 500 == 0:
            print(f"CorLoc is {hits/positives}")
    corloc = hits/positives
    print(f"CorLoc is {corloc}")
    print(f"mean CorLoc is {(corloc).sum()/20}")

CorLoc is [0.5        0.76923077 0.53333333 0.30434783 0.23809524 1.
 0.75609756 0.34482759 0.29411765 0.66666667 0.36       0.625
 0.66666667 0.86363636 0.2393617  0.42105263 0.72727273 0.36363636
 0.60714286 0.30434783]
CorLoc is [0.48       0.56603774 0.52941176 0.23684211 0.27659574 0.78571429
 0.74482759 0.35384615 0.37894737 0.67741935 0.32608696 0.6
 0.61111111 0.74       0.245      0.36842105 0.61904762 0.34042553
 0.56818182 0.37037037]
CorLoc is [0.47761194 0.63157895 0.50485437 0.28571429 0.29166667 0.74468085
 0.72727273 0.41836735 0.37593985 0.74       0.33846154 0.5625
 0.56097561 0.76       0.22128378 0.38095238 0.62857143 0.42424242
 0.55555556 0.45205479]
CorLoc is [0.53488372 0.62       0.46575342 0.30136986 0.31632653 0.71212121
 0.69117647 0.47692308 0.34659091 0.75438596 0.34146341 0.53448276
 0.54867257 0.78350515 0.22487437 0.31958763 0.63636364 0.39583333
 0.57843137 0.48514851]
CorLoc is [0.56637168 0.61788618 0.46629213 0.28888889 0.29365079 0.70238095
 0.6793

CorLoc is [0.6302521  0.57613169 0.40909091 0.32596685 0.25819672 0.66129032
 0.71388499 0.47477745 0.3011236  0.55319149 0.13       0.38004751
 0.6445993  0.74285714 0.2435259  0.33877551 0.57291667 0.28820961
 0.66283525 0.5703125 ]
mean CorLoc is 0.4738992749533109

In [33]:
torch.argmax(cls_scores)

tensor(5, device='cuda:0')

In [32]:
cls_scores

tensor([8.8015e-03, 1.0073e-06, 4.8591e-05, 6.4935e-04, 6.0962e-02, 7.7546e-01,
        1.8202e-07, 6.9581e-03, 5.6128e-05, 1.4432e-06, 7.3070e-05, 4.7705e-04,
        5.4533e-05, 1.5168e-09, 5.5606e-03, 2.2619e-04, 3.8841e-04, 1.5340e-06,
        9.6877e-10, 9.0330e-04, 1.5512e-03, 8.5839e-04, 3.2774e-06, 7.0984e-06,
        1.8416e-06, 1.1724e-09, 4.8746e-05, 1.7280e-02, 6.4380e-06, 1.2533e-02,
        1.1148e-07, 7.8140e-10, 1.2976e-04, 2.9895e-04, 8.0396e-03, 1.5438e-02,
        9.2735e-04, 1.6008e-05, 4.3926e-05, 1.5715e-02, 7.8783e-08, 2.2923e-06,
        3.6833e-09, 1.6496e-07, 1.2807e-09, 2.2752e-08, 6.5740e-07, 1.1553e-03,
        1.6359e-10, 6.9929e-03, 4.5150e-06, 1.2659e-09, 2.5307e-06, 8.9998e-03,
        2.0705e-08, 5.4474e-08, 2.1282e-05, 1.4078e-04, 2.3477e-04, 1.0672e-07,
        2.5046e-05, 1.1427e-04, 4.5559e-03, 2.3163e-04, 8.4948e-08, 2.9120e-06,
        6.4301e-05, 1.3498e-04, 1.2009e-07, 1.6424e-07, 2.8923e-05, 2.8104e-05,
        4.8467e-10, 6.0721e-06, 1.6605e-

In [30]:
hits

array([143, 139, 132,  59,  68, 121, 509, 165, 140,  74,  33, 166, 182,
       184, 464,  82,  60,  70, 172, 143])

In [ ]:
propose_way = "edge_box"
pretrained = "alexnet"
traindata = VOCDectectionDataset("~/data/", 2007, 'test', region_propose=propose_way, do_transform=True)
traindata_loader = data.DataLoader(traindata, 1, shuffle=False)
wsddn = WSDDN_Alexnet().to(DEVICE)
wsddn.load_state_dict(torch.load(SAVE_PATH + get_model_name(propose_way, "2007" , f"wsddn_{pretrained}") + ".pt"))

for epoch in tqdm(range(EPOCHS), "Total"):
    epoch_loss = 0.0
    y_pred = []
    y_true = []

    for img, gt_box, gt_target, regions, scores in tqdm(train_loader, f"Epoch {epoch}"):
        optimizer.zero_grad()
        # img   : Tensor(1, 3, h, w)
        # gt_tar: Tensor(1, R_gt)
        # region: Tensor(1, R, 4)
        img = img.to(DEVICE)
        regions = regions.to(DEVICE)
        gt_target = gt_target.to(DEVICE)
        if propose_way != "edge_box":
            scores = None
        else:
            scores = scores.to(DEVICE)
        combined, fc7 = wsddn(img, regions, scores=scores)

        image_level_cls_score = torch.sum(combined, dim=0) # y
        # sum of combined may create value > 1.0
        image_level_cls_score = torch.clamp(image_level_cls_score, min=0.0, max=1.0)
        reg = alpha * wsddn.spatial_regulariser(regions[0], fc7, combined, gt_target[0])
        loss = bce_loss(image_level_cls_score, gt_target[0])
        out = loss + reg
        
        epoch_loss += out.item()
        out.backward()
        optimizer.step()

        y_pred.append(image_level_cls_score.detach().cpu().numpy().tolist())
        y_true.append(gt_target[0].detach().cpu().numpy().tolist())
    cls_ap = []
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    for i in range(20):
        cls_ap.append(average_precision_score(y_true[:,i], y_pred[:,i]))